# Test Different Evalution Functions
1. Automatically select 10 best generators
2. Generate 20 parameter sets for each generator (directly from generator)
3. Simulate bioreactor to see the final concentration
4. Calculate scoring function and find 3 most fitted ones

In [ ]:
import sys
import os
# Add the parent directory to the PYTHONPATH
sys.path.append(os.path.abspath('../'))


### 2. Generate Parameter Set Using Generators

In [ ]:
import numpy as np
import pandas as pd
from configparser import ConfigParser
from renaissance.evostrat.init_mlp import MLP
from renaissance.tools.helper import load_pkl


In [ ]:
# Load Prameters from the repeat XX generation XX
repeat = 7
generation = 21
cond_class = 1

# 
lnminkm = -25
lnmaxkm = 0

pf_flag = 0
base = "../models/"
met_model = "ecoli_shikki_regulation"
names_km = load_pkl(f'{base}/{met_model}/parameters/generated_k_names.pkl')

n_sets = 10
path_to_weights = f"../output/rnsc_opt/repeat_{repeat}/weights_{generation}.pkl"
output_name = "gen_x_parameters"
ss_idx = 1712

#Parse arguments from configfile
configs = ConfigParser()
configs.read('configfile.ini')

In [ ]:
mlp = MLP(cond_class, lnminkm, lnmaxkm, n_sets, names_km, len(names_km), param_fixing=pf_flag)

# Load saved weights and generate
opt_weights = load_pkl(path_to_weights)
mlp.generator.set_weights(opt_weights)
gen_params = mlp.sample_parameters()

### 3. Load Model and Prepare Simulation

In [ ]:
from pytfa.io.json import load_json_model
from skimpy.core import *
from skimpy.mechanisms import *
from skimpy.sampling.simple_parameter_sampler import SimpleParameterSampler
from skimpy.io.yaml import load_yaml_model
from skimpy.io.regulation import load_enzyme_regulation
from skimpy.analysis.oracle.load_pytfa_solution import load_fluxes, load_concentrations, load_equilibrium_constants
from skimpy.core.parameters import ParameterValues, ParameterValuePopulation
from renaissance.kinetics.eigenvalue_calculation import calc_eigenvalues
import renaissance.tools.helper as hp 
from renaissance.evostrat.init_mlp import MLP
''' Run parameters and paths '''
# Cellular parameters
CONCENTRATION_SCALING = 1e9  # 1 mol to 1 mmol
TIME_SCALING = 1  # 1hr
DENSITY = 1105  # g/L
GDW_GWW_RATIO = 0.3  # Assumes 70% Water
flux_scaling_factor = 1e-3 * (GDW_GWW_RATIO * DENSITY) * CONCENTRATION_SCALING / TIME_SCALING
NCPU = 32

In [ ]:
model_file = configs['PATHS']['model_file']
thermo_experiment_file = configs['PATHS']['thermo_experiment_file']
regulation_file = configs['PATHS']['regulation_file']
kinetic_params_file = configs['PATHS']['kinetic_params_file']
steady_states_file = configs['PATHS']['steady_states_file']
steady_states_sample = 1712 #[1715,1717,2392,2482,3927,4230,4241,4456, 4468]

path_to_kmodel = f'{base}/{met_model}/kinetic/{model_file}'
path_to_tmodel = f'{base}/{met_model}/thermo/{thermo_experiment_file}'
path_to_params = f'{base}/{met_model}/parameters/gen_{repeat}_{generation}_parameters.hdf5'
path_to_regulation_data = f'{base}/{met_model}/{regulation_file}'
path_to_samples = f'{base}/{met_model}/steady_state_samples/{steady_states_file}'


In [ ]:
# Load models
tmodel = load_json_model(path_to_tmodel)
kmodel_draft = load_yaml_model(path_to_kmodel)

# Add regulation data to kinetic model
df = pd.read_csv(path_to_regulation_data)
df_regulations_all = df[df['reaction_id'].isin(list(kmodel_draft.reactions.keys()))]
df_regulations_all = df_regulations_all[df_regulations_all['regulator'].isin(list(kmodel_draft.reactants.keys()))]
kmodel = load_enzyme_regulation(kmodel_draft, df_regulations_all)

# Compile model
kmodel.prepare()
kmodel.compile_jacobian(ncpu=NCPU)

In [ ]:
ss_idx = steady_states_sample 
samples = pd.read_csv(path_to_samples, header=0, index_col=0).iloc[ss_idx, 0:]

flux_series = load_fluxes(samples, tmodel, kmodel,
                  density=DENSITY,
                  ratio_gdw_gww=GDW_GWW_RATIO,
                  concentration_scaling=CONCENTRATION_SCALING,
                  time_scaling=TIME_SCALING)

conc_series = load_concentrations(samples, tmodel, kmodel,
                          concentration_scaling=CONCENTRATION_SCALING)
    
# Fetch equilibrium constants
k_eq = load_equilibrium_constants(samples, tmodel, kmodel,
                          concentration_scaling=CONCENTRATION_SCALING,
                          in_place=True)


sampling_parameters = SimpleParameterSampler.Parameters(n_samples=1)
sampler = SimpleParameterSampler(sampling_parameters)
sampler._compile_sampling_functions(kmodel, conc_series)


In [ ]:
parameter_set = pd.DataFrame(np.exp(gen_params))
parameter_set.columns = names_km

param_population = []
for j in range(len(parameter_set.index)):
    param_val = parameter_set.loc[j] * CONCENTRATION_SCALING
    param_sample, eigen = calc_eigenvalues(kmodel, param_val, flux_series, conc_series, k_eq)
    param_population.append(param_sample)

param_population = ParameterValuePopulation(param_population, kmodel=kmodel)

In [ ]:
gen_params

### 3. Bioreactor Simulation

In [ ]:
from skimpy.simulations.reactor import make_batch_reactor
from renaissance.simulation.reactor import simulate_bioreactor
import time

# Ode simulation parameters
TOTAL_TIME = 60
N_STEPS = 1000
MAX_TIME = 60
REACTOR_VOLUME = 50e13

In [ ]:
# Reactor initialization
kinetic_models = list(param_population._index.keys())
reactor = make_batch_reactor('../2_bioreactor/single_species.yaml', df_regulation= df_regulations_all)
reactor.compile_ode(ncpu=NCPU, add_dilution=False)
reactor_volume = reactor.models.strain_1.parameters.strain_1_volume_e.value

In [ ]:
sols_wt= []
finals = {}
for this_model in kinetic_models:
    print("\nRunning model: {}".format(this_model))

    # Load tfa sample and kinetic parameters into kinetic model
    parameter_set = param_population[this_model]
        
    solution, final_biomass, final_anthranilate, final_glucose = \
    simulate_bioreactor(parameter_set, 
                        samples, 
                        kmodel, 
                        conc_series, 
                        reactor, 
                        reactor_volume=REACTOR_VOLUME,
                        max_time=MAX_TIME,
                        total_time=TOTAL_TIME,
                        steps=N_STEPS)
    finals[this_model] = [final_biomass, final_anthranilate, final_glucose]
    sols_wt.append(solution)


### 4. Calculate Scores

In [ ]:
import matplotlib.pyplot as plt

# Load Experimental Value
growth = pd.read_csv('../data/experimental_data/biomass_wt.csv')
glc = pd.read_csv('../data/experimental_data/glc_wt.csv')
anth = pd.read_csv('../data/experimental_data/anth_wt.csv')
glc[glc.columns[1]] -= 0.54
glc[glc.columns[2]] -= 0.54
glc[glc.columns[3]] -= 0.54
dict_scaling = {'biomass_strain_1': 0.28e-12/0.5, 
                'anth_e': 136.13 * 1e-9, 
                'glc_D_e': 1e-9 * 180.156}
labels = {'biomass_strain_1': 'biomass (g)', 'anth_e': 'anthranilate (g/L)', 'glc_D_e': 'glucose (g/L)'}
exp_data = {'biomass_strain_1': growth, 'anth_e': anth, 'glc_D_e': glc}
T = np.linspace(0, TOTAL_TIME, N_STEPS)

In [ ]:
n_simulations = 10
all_sols = sols_wt
plt.figure(figsize = (10,25))
plt.title('Generation', fontsize = 50)
ix=1
final_biomass = []
final_anth = []
final_glc = []

for conc, scaling in dict_scaling.items():
    
    plt.subplot(5,1,ix)

    # Plot simulated data

    for sol_id in range(n_simulations):
        this_sol = all_sols[sol_id].concentrations
        if this_sol.shape[0]==N_STEPS:
            plt.plot(T, this_sol[conc]*scaling, color = 'green', alpha = 0.5)
        if conc== 'biomass_strain_1': final_biomass.append(list(this_sol[conc]*scaling)[-1])
        if conc== 'anth_e':  final_anth.append(list(this_sol[conc]*scaling)[-1])
        if conc== 'glc_D_e':  final_glc.append(list(this_sol[conc]*scaling)[-1])
        
    # Plot experimental data (when available)
    
    exp_ = exp_data[conc]
    mean = exp_[exp_.columns[1]]
    time_exp = exp_[exp_.columns[0]]

    if len(exp_.columns) > 2:
        lo = exp_[exp_.columns[2]]
        hi = exp_[exp_.columns[3]]
        plt.errorbar(time_exp, mean, yerr=np.asarray([mean - lo, hi - mean]),
                     fmt='ko', label= f'{conc}', capsize=5)
    else:
        plt.plot(time_exp, mean, 'ko', label=f'{conc}')  

    if conc== 'biomass_strain_1': plt.title(f'max biomass: {np.round(np.max(final_biomass),4)}')
    
    else: plt.title(f'{conc}')

    
    plt.xlabel('time (h)')
    plt.ylabel(labels[conc])
    plt.xlim([0, 60])
    ix+=1
  

#plt.savefig(f'{plot_output}/bioreactor_results.jpg',dpi = 300, bbox_inches = 'tight')
plt.show()
plt.close()



In [ ]:
# Function to calculate NRMSE
def nrmse_score(errors, scaling_factor):
    nrmse = 1 / (np.exp(errors / scaling_factor))**2
    return nrmse


In [ ]:
def time_stamp_matching(N_STEPS, TOTAL_TIME, current_time):
    if current_time > TOTAL_TIME:
        return N_STEPS
    else:
        return int(current_time * (N_STEPS-1) / TOTAL_TIME)

In [ ]:
n_last_points = 1

all_solutions = exp_data.copy()
total_reward = 0
for conc, scaling in dict_scaling.items():
    df = all_solutions[conc][['Time', 'mean']].iloc[-n_last_points:].copy()
    df.rename(columns={'mean':'experimental'}, inplace=True)
    
    temp_df = pd.DataFrame()
    for i in range(10):
        sim_conc = all_sols[i].concentrations[conc]

        if sim_conc.shape[0]==N_STEPS:
            new_df = df.copy() 
            new_df['simulated'] = new_df['Time'].apply(lambda x: sim_conc[time_stamp_matching(N_STEPS, TOTAL_TIME, x)]*scaling)
            new_df['sq_err'] = (new_df['experimental'] - new_df['simulated']) ** 2
            new_df['reward'] = nrmse_score(new_df['sq_err'], 0.5)
            temp_df = temp_df.append(new_df, ignore_index=True)  # Append to temp_df
    all_solutions[conc] = temp_df

    total_reward += temp_df['reward'].mean() / 3


In [ ]:
total_reward

In [ ]:
from permetrics import RegressionMetric

last_n_points = 5
evaluator = RegressionMetric()

full_score = {}
for i in range(n_simulations):
    sum = 0
    for conc, scaling in dict_scaling.items():
        sim_conc =all_sols[i].concentrations[conc]
        if sim_conc.shape[0]==N_STEPS:
            time_points = exp_data[conc]['Time'][-last_n_points:].values
            exp_final_value = exp_data[conc]['mean'][-last_n_points:].values
            sampling_time = [time_stamp_matching(N_STEPS, TOTAL_TIME, t) for t in time_points]
            sim_final_value = sim_conc[sampling_time].values*scaling
            score = nrmse_score(sim_final_value, exp_final_value, np.max(exp_final_value))
            print(f'RMSE of model {i}: {conc}: {score}')
            sum += score
    full_score[i] = sum

In [ ]:
from renaissance.evostrat.rewards import check_simulated_RMSE
check_simulated_RMSE(all_sols, exp_data, dict_scaling, N_STEPS, TOTAL_TIME, last_n_points)